In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from sklearn.decomposition import PCA
from gensim.models.doc2vec import Doc2Vec
import os
import re

import sys
sys.path.append('utils')
import labels
import plots 
import interpret
import accuracy
from accuracy import pairwise_accuracy
from interpret import Interpret

#import utils.labels as labels
#import utils.plots as plots
#from utils.interpret import Interpret
#from utils.accuracy import pairwise_accuracy

## Figure 3- Party Placement in Canada

In [7]:
# load in Canada model

canmodel = Doc2Vec.load('model/updated_model')

can_dict = labels.party_labels('Canada')
cannames, canparties, cancols, canmkers = labels.party_tags(canmodel, 'Canada')

# preprocess the keys in canparties to have it match the inverted can_dict
#canparties_processed = [party.replace('_', ' ') for party in canparties]

# invert the dictionary to map from values to keys
can_dict_inverted = {value: key for key, value in can_dict.items()}

# changing this so that it only looks at ones that are actually in the dictionary
canlabs = [can_dict_inverted[p] for p in canparties if p in can_dict_inverted]
Mcan = canmodel.vector_size
#Pcan = len(canparties_processed)
Pcan = len(canparties)


In [8]:
zcan = np.zeros((Pcan, Mcan))
for i in range(Pcan):
    zcan[i,:] = canmodel.docvecs[canparties[i]]
pca_can = PCA(n_components = 2)
Zcan = pd.DataFrame(pca_can.fit_transform(zcan), columns = ['dim1', 'dim2'])

# GRACE NEED TO GET LABELS TO WORK
Zcan['label'] = canparties

/var/folders/r_/rw1ty4x55gl73b77mkvn_bkc0000gn/T/ipykernel_31896/2325840444.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  zcan[i,:] = canmodel.docvecs[canparties[i]]


In [9]:
Zcan

,dim1,dim2,label
0,8.263308,-9.224139,New Democratic Party_2013
1,8.472222,-9.399086,New Democratic Party_2014
2,5.877826,-10.291950,New Democratic Party_2015
3,7.701108,-8.616850,New Democratic Party_2012
4,-5.111331,-11.023739,New Democratic Party_1988
...,...,...,...
509,-11.894556,-2.139780,Conservative_2005
510,-8.705391,0.334099,Progressive Conservative_2002
511,-8.015775,0.937191,Progressive Conservative_2003
512,-9.517301,-1.676125,Conservative_2004


In [34]:
# change all instances of 'New democratic party' to NDP

Zcan['label'] = Zcan['label'].apply(lambda x: re.sub(r'New Democratic Party', 'NDP', x))
Zcan['label'] = Zcan['label'].apply(lambda x: re.sub(r'Conservative', 'Cons', x))
Zcan['label'] = Zcan['label'].apply(lambda x: re.sub(r'Bloc Québécois', 'Bloc', x))
# had to do my own research to find out that reform party/canadian alliance merged with the progressive conservative party
Zcan['label'] = Zcan['label'].apply(lambda x: re.sub(r'Progressive Cons', 'RefAll', x))

In [27]:
# Re-orienting the first axis for substantive interpretation:
if Zcan[Zcan.label=='NDP_2015'].dim1.values[0] > Zcan[Zcan.label=='Cons_2015'].dim1.values[0]:
    Zcan['dim1'] = Zcan.dim1 * (-1)

In [28]:
plots.plot_3b(Zcan, canlabs, cancols, canmkers, savepath='figures/figure3b.pdf')
print("Saved Figure 3b to file figures/figure3b.pdf")

Saved Figure 3b to file figures/figure3b.pdf
